# CAPEX PROJECT EVENTS 

### Build Appended files with [Abandoned, Completed and Outstanding] project status CMIE Capex (2014-22)

#####  Create and Store Long Form Data from raw Data

In [33]:
print("INIZIO - LONG FORM - PROJECT EVENTS")

### pick Project events file in each of  [Abandoned, Completed and Outstanding]  folder and append to proc_data folder

import os
import pandas as pd
import re

folder_list = ['Completed', 'Abandoned', 'Outstanding']

# List to store data from all XLSX files
data = []

for stub in folder_list: 
    # Parent folder path
    parent_folder = f'/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/raw_data/{stub}/'
    #print('Level:0 ' + stub + parent_folder)

    # Iterate through files in the folder
    for filename in os.listdir(parent_folder):
        if filename.endswith('_Events_total.xlsx'):
            #print(filename)
            file_path = os.path.join(parent_folder, filename)

            # Read XLSX file into a DataFrame
            df = pd.read_excel(file_path)

            # Append the DataFrame to the data list
            data.append(df)
        else :
            print("Events data not found")

# Concatenate all DataFrames in the data list
events_df = pd.concat(data)
events_df.drop(['FY','Quarter', 'Quarter '], axis=1, inplace = True)
# Remove rows where company code , project number and Event info is missing.
events_df.dropna(subset=['Company Code','Project No.','Event','Date'], inplace = True )

# Remove rows having event info as "Company contacted but no response","Company contacted successfully"
excl_list = ["Company contacted but no response","Company contacted successfully"]
events_df = events_df[~events_df.Event.str.contains('|'.join(excl_list))]
events_df = events_df[~(events_df['Event']=="Event")]

# Convert String to Date
events_df['Date2']= pd.to_datetime(events_df['Date'])
events_df.drop(columns=['Date'], inplace=True)
events_df.rename(columns={'Date2': 'Date'}, inplace=True)

#### merge with event concordance table to get project stage IDs and substage IDs
concord_folder = f'/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/raw_data/'
df_event_hierarchy = pd.read_excel(os.path.join(concord_folder, 'Concordance-Tables-Capex.xlsx'), sheet_name = 'events-concord') # Events (Substage) Alphanumeric code numeric code and region

merged_df = pd.merge(events_df, 
                     df_event_hierarchy[['Event','sub_stage_id','stage_id']],  # removed stage_id
                     on='Event', how='left')

#### Prepare Long and Wide format data and store in excel files

# Group and create date based ascending order for project events
group_cols = ['Company Name', 'Project Name','Company Code', 'Project No.']
# Group by 'GroupColumn' and Sort by ascending 'Date'
merged_df = merged_df.groupby( group_cols,group_keys=False).apply(lambda group: group.sort_values(by='Date'))

### Append All event information into a New column 
# Create a new column 'Appended_sub_stage_id' with appended 'sub_stage_id' values by 'Name'
merged_df['Append_stage_id'] = merged_df.groupby(group_cols)['stage_id'].transform(lambda x: ''.join(x))
merged_df['Append_sub_stage_id'] = merged_df.groupby(group_cols)['sub_stage_id'].transform(lambda x: ''.join(x))

# Function to reorder characters in a string alphabetically
def reorder_alphabetically(text):
    return ''.join(sorted(text))

# Apply the function to the 'TextColumn' and create a new column 'ReorderedText'
merged_df['Append_stage_id'] = merged_df['Append_stage_id'].apply(reorder_alphabetically)


### Retain data to Long Format - But remove duplicate event records
unique_id_list = ['Company Name', 'Project Name', 'Cost (Rs.million)', 
                  'Project Status', 'Last Updated On', 'Event', 
                  'Company Code', 'Project No.', 'Type', 'sub_stage_id', 'stage_id',]

# Generate a serial counter within each group
merged_df['Proj_Event_Counter'] = merged_df.groupby(unique_id_list).cumcount() + 1

df_long = merged_df[merged_df['Proj_Event_Counter']<=1]

df_long.drop(columns=['Proj_Event_Counter'], axis =1, inplace = True)

# Save the LONG DataFrame into Excel
print('LONG')
out_folder_path = f'/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/proc_data/'
out_file_name_1 = 'Capex_Events_Total_L.xlsx'
out_file_path_1 = os.path.join(out_folder_path, out_file_name_1)
df_long.to_excel((out_file_path_1), index=False)
print(df_long.shape)
print(df_long.columns)


print('FINITO - LONG FORM')


INIZIO - LONG FORM - PROJECT EVENTS
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found
Events data not found


/var/folders/4h/cjm_23314cv7n8szb95hx4nc0000gn/T/ipykernel_2244/2351940458.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_long.drop(columns=['Proj_Event_Counter'], axis =1, inplace = True)


LONG
(137808, 14)
Index(['Company Name', 'Project Name', 'Cost (Rs.million)', 'Project Status',
       'Last Updated On', 'Event', 'Company Code', 'Project No.', 'Type',
       'Date', 'sub_stage_id', 'stage_id', 'Append_stage_id',
       'Append_sub_stage_id'],
      dtype='object')
FINITO - LONG FORM


### Convert to Wide Format

In [53]:
print("INIZIO - WIDE FORM - PROJECT EVENTS")

########## Convert data to Wide Format ######

master_list = df_long.columns
col_list = ['sub_stage_id'] # column whose row values will be appended to column heading 
val_list = ['Date', 'Event', 'stage_id' ]
idx_list = [item for item in master_list if item not in col_list and item not in val_list]
print(idx_list)

# Pivot the DataFrame to wide format
df_wide = df_long.pivot(index = idx_list, columns = col_list, values = val_list).reset_index()

# Flatten the multi-level column index
df_wide.columns = [f'{col[0]}_{col[1]}' if isinstance(col, tuple) else col for col in df_wide.columns]

# Remove trailing underscores from column headings
df_wide.columns = df_wide.columns.str.rstrip('_')

# Generate FY and Quarter Info to enable matching 
# Function to generate fiscal year column in 'YY-YY' format
def gen_FY(date):
    year = date.year
    fiscal_year = f'{str(year)[-2:]}-{str(year+1)[-2:]}'
    return fiscal_year

# Function to generate Indian Fiscal Quarters 
def gen_FY_Q(date):
    fiscal_year = date.year
    fiscal_year_last_digit = fiscal_year % 100  # Get the last two digits of the year
    if date.month >= 4:
        quarter = ((date.month - 4) // 3) + 1
    else:
        quarter = 4  # Quarter 4 of the previous fiscal year
        fiscal_year -= 1
#     fiscal_year_quarter = f'FY-{fiscal_year_last_digit:02d}-Q{quarter}'
    return quarter


# Apply the functions to create new columns
df_wide['FY'] = df_wide['Date_A1'].apply(gen_FY)
df_wide['Quarter'] = df_wide['Date_A1'].apply(gen_FY_Q)

#### Create Elapsed Time Variables
# Function to create a new column based on string match
def last_date(row):
    if 'Abandoned' in row['Type']:
        selected_date_columns = ['Date_H19', 'Date_H20', 'Date_H21', 'Date_H22']
    elif 'Completed' in row['Type']:
        selected_date_columns = ['Date_H9']
    elif 'Outstanding' in row['Type']:
        selected_date_columns = df_wide.filter(regex='^Date_').columns.tolist()
    else:
        return None
    
    selected_dates = row[selected_date_columns]
    
    # Filter out missing values before finding the maximum
    selected_dates = selected_dates.dropna()
    
    if selected_dates.empty:
        return None
    
    return selected_dates.max()

# Apply the function to create a new column 'Last_Date'
df_wide['Last_Date'] = df_wide.apply(last_date, axis=1)
df_wide['Elapsed_time'] = df_wide['Last_Date']-df_wide['Date_A1']

# Convert the 'Quarter' column in df_wide to int64 to enable merge
df_wide['Quarter'] = df_wide['Quarter'].astype(int)

print(df_wide.shape)
print(df_wide.columns)

print("INTERVALLO - WIDE FORM")


INIZIO - WIDE FORM - PROJECT EVENTS
['Company Name', 'Project Name', 'Cost (Rs.million)', 'Project Status', 'Last Updated On', 'Company Code', 'Project No.', 'Type', 'Append_stage_id', 'Append_sub_stage_id']


/var/folders/4h/cjm_23314cv7n8szb95hx4nc0000gn/T/ipykernel_2244/1363871242.py:12: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_wide = df_long.pivot(index = idx_list, columns = col_list, values = val_list).reset_index()
/var/folders/4h/cjm_23314cv7n8szb95hx4nc0000gn/T/ipykernel_2244/1363871242.py:12: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_wide = df_long.pivot(index = idx_list, columns = col_list, values = val_list).reset_index()
/var/folders/4h/cjm_23314cv7n8szb95hx4nc0000gn/T/ipykernel_2244/1363871242.py:12: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df_wide = df_long.pivot(index = idx_list, columns = col_list, values = val_list).reset_index()


(22255, 278)
Index(['Company Name', 'Project Name', 'Cost (Rs.million)', 'Project Status',
       'Last Updated On', 'Company Code', 'Project No.', 'Type',
       'Append_stage_id', 'Append_sub_stage_id',
       ...
       'stage_id_H7', 'stage_id_H8', 'stage_id_H9', 'stage_id_I3',
       'stage_id_I4', 'stage_id_I7', 'FY', 'Quarter', 'Last_Date',
       'Elapsed_time'],
      dtype='object', length=278)
INTERVALLO - WIDE FORM


##### Merge with Final_Capex Excel File to get product, state and district location and Industry group information

In [54]:

### Write Merged file to Excel 
in_folder_path_2 = '/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/proc_data/'
in_file_name_2 = 'Final_Capex.xlsx'
df_proj_loc = pd.read_excel(os.path.join(in_folder_path_2, in_file_name_2))
# print(df_tmp.dtypes)

print('Proj.Loc. & SGDP Shape, columns-B4: \n', df_proj_loc.shape)

### Remove duplicate project records. Retain first Location (District) etc. record.
common_columns = ['Company Name', 'Project Name',
                  'Project Status','Company Code', 'Project No.', 'Type', 'FY', 'Quarter' ]

# Generate a serial counter within each group
df_proj_loc['Proj_Counter'] = df_proj_loc.groupby(common_columns).cumcount() + 1

df_proj_loc = df_proj_loc[df_proj_loc['Proj_Counter']==1]
print(df_proj_loc['Year'].unique())
df_proj_loc.drop(columns=['Proj_Counter'], axis =1, inplace = True)

print('Proj.Loc.& SGDP Shape, columns: \n', df_proj_loc.shape)
print(df_proj_loc.columns)

# # merge_events_proj = pd.merge(df_wide, 
# #                      df_tmp[['Company Name', 'Project Name', 'Project Status','Company Code', 
# #                              'Project No.', 'Type', 'FY', 'Quarter', 
# #                              'Location', 'District', 'st_ut_code', 'region',
# #                              'Sector', 'Owner Affliation', 'Ownership Group', 'Foreign', 
# #                              'Industry Group', 'IndG_L0', 'IndG_L4', 'IndG_L8']],
# #                      on=common_columns,
# #                      how='left')
merge_events_proj = pd.merge(df_wide, 
                     df_proj_loc,
                     on=common_columns,
                     how='left')
print("Merged Data Shape & Columns: \n", merge_events_proj.shape)
print(merge_events_proj.columns)
# Save the WIDE DataFrame into Excel
print('WIDE')
out_folder_path = f'/Users/kalyan/Library/CloudStorage/OneDrive-IIMVIZAG/Python-Exercise-KK/Kalyan-Jupyter-Notebooks/Capex/proc_data/'
out_file_name_2 = 'Capex_Events_Total_W.xlsx'
out_file_path_2 = os.path.join(out_folder_path, out_file_name_2)
merge_events_proj.to_excel((out_file_path_2), index=False)


print("FINITO - WIDE FORM")


Proj.Loc. & SGDP Shape, columns-B4: 
 (81197, 54)
[2018 2017 2020 2014 2022 2015 2021 2016 2019]
Proj.Loc.& SGDP Shape, columns: 
 (32409, 54)
Index(['Company Name', 'Project Name', 'Cost (Rs.million)', 'Project Status',
       'Last Updated On', 'Industry Group', 'Ownership Group', 'Project Type',
       'Company Code', 'Project No.', 'FY', 'Quarter', 'Type', 'Location',
       'District', 'State', 'Products', 'Capacity', 'Units',
       'Reason for Stalling of Projects', 'Sector', 'Owner Affliation',
       'Foreign', 'st_ut_code', 'region', 'IndG_L0', 'IndG_L4', 'IndG_L8',
       'Gross Fiscal Deficit', 'Revenue Deficit', 'Primary Deficit',
       'Own Tax Revenue', 'Own Non-Tax Revenue', ' Interest Payments',
       'State-wise Pension', 'State-wise Capital Expenditure',
       'Capital Outlay', 'Social Sector Expenditure',
       ' Gross State Domestic Product', 'GSVA-Agriculture',
       'GSVA-Manufacturing', 'GSVA-Construction', 'GSVA-Industry',
       'GSVA-Banking and Insuranc

In [56]:
print(merge_events_proj['FY'].unique())

['22-23' '21-22' '18-19' '15-16' '19-20' '20-21' '17-18' '16-17' '23-24'
 '14-15']


In [58]:
print(merge_events_proj['Quarter'].unique())

[1 2 4 3]
